In [40]:
import scipy.stats as st
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

In [41]:
data = pd.read_csv("data/nba.csv")
data[:10]

,Unnamed: 0,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
0,7,A. Johnson,ATL,30,34,24,28,64,52
1,10,A. Johnson,TOR,63,57,44,38,120,82
2,15,Abdur-Rahim,SAC,114,124,84,90,238,174
3,22,Afflalo,DEN,236,284,189,227,520,416
4,23,Afflalo,DET,75,88,62,67,163,129
5,33,Aldridge,POR,899,940,678,748,1839,1426
6,35,Alexander,MIL,36,47,24,34,83,58
7,36,Allen,BOS,246,237,211,205,483,416
8,40,Allen,MEM,212,186,158,146,398,304
9,45,Allen,SEA,151,156,137,140,307,277


Разницу в успехе бросков у игроков в домашних и гостевых играх можно понимать как разницу в долях успешных бросков.

Для каждого игрока имеем 2 несвязанные выборки $X_i = (X_{1i},\dots,X_{ni}),~X_{ij}\sim Be(p_{i1}), ~~Y_i = (Y_{1i},\dots,Y_{ni}),~Y_{ij} \sim Be(p_{i2})$ можем проверить гипотезу 

$H_0: p_{i1} = p_{i2}$

$H_1: p_{i1} \ne p_{i2}$


Используем $Z-$критерий для проверки равенства долей 2-х независимых выборок для каждого игрока.

Так как происходит множественная проверка гипотез, сделаем поправку

Просуммируем строки по игрокам:

In [42]:
df = data.groupby('player').agg(sum)
print(f'{len(df)} players')

458 players


Без поправки:

In [43]:
p_vals = []
for _, player in df.iterrows():
    p_vals.append(
        proportions_ztest(
            (player['score_away'], player['score_home']),
            (player['atm_away'], player['atm_home']), 
            alternative="two-sided"
        )[1]
    )
p_vals = np.array(p_vals)

print(f'Без поравки отвергается гипотез: {(p_vals < 0.05).sum()}')

Без поравки отвергается гипотез: 32


C поправкой:

In [44]:
print(f"Без поравки отвергается гипотез: {multipletests(p_vals, method='bonferroni')[0].sum()}")

Без поравки отвергается гипотез: 1
